#### download package

In [8]:
# download sktime package 
!pip install sktime

In [9]:
# 필요한 패키지 import
import os
import sys
import warnings
import plotly
import numpy as np
import pandas as pd
import datetime
import tensorflow as tf
from tqdm import tqdm
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold, train_test_split, TimeSeriesSplit, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn import preprocessing
from sklearn.preprocessing import RobustScaler, StandardScaler
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.utils.plotting import plot_series
from scipy.stats import reciprocal 

from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from sklearn.svm import SVR
import lightgbm as lgb
from lightgbm.sklearn import LGBMRegressor
from sklearn.multioutput import MultiOutputRegressor

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.5f}'.format

## Model training

### making datasets

In [12]:
cluster_df = ['0.0','1.0','2.0','3.0','4.0','5.0','6.0','7.0','8.0','9.0','10']

In [13]:
# 학습에 필요한 train 및 test dataset 만드는 과정
def get_train_test_set(cluster):
  df = pd.read_pickle("cluster_"+cluster+".pkl")
  train = df[df['BKG_DATE'] <= '2021-06-20']
  test = df[df['BKG_DATE'] > '2021-06-20']
  # 나머지 Scaling
  scaling_features = ['DAY_1', 'DAY_2', 'DAY_3', 'WEEK_AMT',
       '100이상 27307.5미만','27307.5이상 63200미만','63200이상 133375미만', '133375이상 290100미만',
       '290100이상 1170901미만','MEAN_PRICE', '강원', '경기', '경남', '경북', '광주',
       '대구', '대전', '부산', '서울', '세종', '울산', '인천', '전남', '전북', '제주', '충남', '충북']

  scaler = StandardScaler()
  train.loc[:, scaling_features] = scaler.fit_transform(train[scaling_features])
  test.loc[:, scaling_features] = scaler.transform(test[scaling_features])
  train_x = train.drop(['ITEM_QTY','BKG_DATE'], axis=1)
  train_y = train['ITEM_QTY']

  test_x = test.drop(['ITEM_QTY','BKG_DATE'], axis=1)
  test_y = test['ITEM_QTY']
  return train_x, train_y, test_x, test_y

### 모델 별 정의 및 파라미터 최적화

In [14]:
# 모델별 학습 파라미터 정의
XGBRegressor_param = {'n_estimators' : [100], 'eta' : [0.01], 'min_child_weight' : np.arange(1, 8, 1), 'max_depth' : np.arange(3,9,1) , 'colsample_bytree' :np.arange(0.8, 1.0, 0.1), 'subsample' :np.arange(0.8, 1.0, 0.1)}
LGBMRegressor_param = {'max_depth' : range(3,15,3), 'min_child_weight': range(1,6,2), 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100], 'learning_rate':[0.1, 0.01], 'max_depth' : [6,8,10]}
SVR_param = {'kernel':['linear'], 'C':[1.0], 'epsilon':[0.1]}
GradientBoostingRegressor_param = {'n_estimators':[100], 'max_depth':np.arange(3,20,3)}
AdaBoostRegressor_param = {'n_estimators' : np.arange(25, 100, 25), 'loss': ['linear', 'square', 'exponential'], 'learning_rate': np.arange(0.1, 1)} 

In [15]:
# 모델별 정의
XGBRegressor_model = XGBRegressor(n_estimators = 100, objective = 'reg:squarederror')
LGBMRegressor_model = LGBMRegressor(n_estimators = 80)
SVR_model = SVR(kernel='linear', C=1.0, epsilon=0.1)
GradientBoostingRegressor_model = GradientBoostingRegressor(n_estimators=100, max_depth=3)
AdaBoostRegressor_model = AdaBoostRegressor(base_estimator=None)

In [16]:
# 단일 모델별 최적 파라미터로 모델링
def print_best_params(model, params, x_train, x_test, y_train, y_test, log=False):

  tss = TimeSeriesSplit(n_splits=5)
  grid_model = GridSearchCV(model, cv = tss, param_grid=params, scoring='neg_mean_absolute_error')
  grid_model.fit(x_train, y_train)
  mae = -1 * grid_model.best_score_
  #print('{0} 최적 평균 mae값 : {1}, 최적 파라미터:{2}'.format(model.__class__.__name__, np.round(mae, 4), grid_model.best_params_))

  best_model = grid_model.best_estimator_
  pred = best_model.predict(x_test)

  if log:
    y_test = np.expm1(y_test)
    pred = np.expm1(pred)
  
  single_min_list = np.round(mean_absolute_error(y_test, pred), 4)

  return best_model, single_min_list, pred

In [17]:
# 단일 모델에서의 MAE 값이 가장 작은 세 개의 모델로 stacking, stacking model의 dataset 만드는 함수
def get_stacking_base_datasets(model, x_train_n, y_train_n, x_test_n, n_splits=5):
  # 지정된 n_folds 값으로 KFold 생성
  tss = TimeSeriesSplit(n_splits)

  # 추후 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화
  train_fold_pred = np.zeros((x_train_n.shape[0], 1))
  test_pred = np.zeros((x_test_n.shape[0], n_splits))
  #print(model.__class__.__name__, ' model 시작')

  for folder_counter, (train_index, valid_index) in enumerate(tss.split(x_train_n)):
    # 입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 세트 추출
    #print('\t 폴드 세트: ', folder_counter, ' 시작')
    x_tr = x_train_n[train_index]
    y_tr = y_train_n[train_index]
    x_te = x_train_n[valid_index]

    # 폴드 세트 내부에서 다시 만들어진 학습 데이터로 기반 모델의 학습 수행
    model.fit(x_tr, y_tr)
    # 폴드 세트 내부에서 다시 만들어지 검증 데이터로 기반 모델 예측 후 데이터 저장
    train_fold_pred[valid_index, :] = model.predict(x_te).reshape(-1, 1)
    # 입력된 원본 테스트 데이터를 폴드 세트내 학습된 기반 모델에서 예측 후 데이터 저장
    test_pred[:, folder_counter] = model.predict(x_test_n)

  # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성
  test_pred_mean = np.mean(test_pred, axis=1).reshape(-1, 1)

  # train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터, test_pred_mean은 테스트 데이터
  return train_fold_pred, test_pred_mean

### 모델 학습 과정
    - 단일 모델 학습 및 stacking 모델 학습
    - 모델별 성능 비교 후 최적의 모델 반환

In [18]:
import sys
mod = sys.modules[__name__]

In [25]:
# 최적의 파라미터로 각각의 모델 학습
def get_optimal_model(cluster):
  x_train, y_train, x_test, y_test = get_train_test_set(cluster)
  x_train_n=x_train.values
  x_test_n=x_test.values
  y_train_n=y_train.values
  
  # 단일 모델별 성능 리스트
  single_min_list = dict()
  
  # 단일 모델 학습
  globals()["XGBRegressor_model_tuned_{}".format(cluster)], single_min_list["XGBRegressor"], globals()["single_pred_XGBRegressor_{}".format(cluster)] = print_best_params(XGBRegressor_model, XGBRegressor_param, x_train, x_test, y_train, y_test)
  globals()["LGBMRegressor_model_tuned_{}".format(cluster)], single_min_list["LGBMRegressor"], globals()["single_pred_LGBMRegressor_{}".format(cluster)] = print_best_params(LGBMRegressor_model, LGBMRegressor_param, x_train, x_test, y_train, y_test)
  globals()["SVR_model_tuned_{}".format(cluster)], single_min_list["SVR"], globals()["single_pred_SVR_{}".format(cluster)] = print_best_params(SVR_model, SVR_param, x_train, x_test, y_train, y_test)
  globals()["GradientBoostingRegressor_model_tuned_{}".format(cluster)], single_min_list["GradientBoostingRegressor"], globals()["single_pred_GradientBoostingRegressor_{}".format(cluster)] = print_best_params(GradientBoostingRegressor_model, GradientBoostingRegressor_param, x_train, x_test, y_train, y_test)
  globals()["AdaBoostRegressor_model_tuned_{}".format(cluster)], single_min_list["AdaBoostRegressor"], globals()["single_pred_AdaBoostRegressor_{}".format(cluster)] = print_best_params(AdaBoostRegressor_model, AdaBoostRegressor_param, x_train, x_test, y_train, y_test)

  single_model_mae = sorted(single_min_list.items(), key = lambda item: item[1])
  
  # Stacking 모델별 성능 리스트
  stacking_list = dict()

# stacking model dataset 생성
  globals()["{}_train_{}".format(single_model_mae[0][0], cluster)], globals()["{}_test_{}".format(single_model_mae[0][0], cluster)] = get_stacking_base_datasets(getattr(mod, "{}_model_tuned_{}".format(single_model_mae[0][0], cluster)), x_train_n, y_train_n, x_test_n, 5)                                                                                                          
  globals()["{}_train_{}".format(single_model_mae[1][0], cluster)], globals()["{}_test_{}".format(single_model_mae[1][0], cluster)] = get_stacking_base_datasets(getattr(mod, "{}_model_tuned_{}".format(single_model_mae[1][0], cluster)), x_train_n, y_train_n, x_test_n, 5) 
                                                                                                            
  # 첫번째 경우
  stack_final_x_train = np.concatenate((getattr(mod, "{}_train_{}".format(single_model_mae[0][0], cluster)), getattr(mod, "{}_train_{}".format(single_model_mae[1][0], cluster))), axis=1)
  stack_final_x_test = np.concatenate((getattr(mod, "{}_test_{}".format(single_model_mae[0][0], cluster)), getattr(mod, "{}_test_{}".format(single_model_mae[1][0], cluster))), axis=1)

  globals()["meta_model_{}_{}".format(single_model_mae[2][0], cluster)] = getattr(mod, "{}_model_tuned_{}".format(single_model_mae[2][0], cluster))

  getattr(mod, "meta_model_{}_{}".format(single_model_mae[2][0], cluster)).fit(stack_final_x_train, y_train)
  globals()["stack_pred_{}_{}".format(single_model_mae[2][0], cluster)] = getattr(mod, "meta_model_{}_{}".format(single_model_mae[2][0], cluster)).predict(stack_final_x_test)
  stacking_list["{}".format(single_model_mae[2][0])] =  mean_absolute_error(y_test, getattr(mod, "stack_pred_{}_{}".format(single_model_mae[2][0], cluster)))

  # 두번째 경우
  globals()["{}_train_{}".format(single_model_mae[0][0], cluster)], globals()["{}_test_{}".format(single_model_mae[0][0], cluster)] = get_stacking_base_datasets(getattr(mod, "{}_model_tuned_{}".format(single_model_mae[0][0], cluster)), x_train_n, y_train_n, x_test_n, 5)
                                                                                                            
  globals()["{}_train_{}".format(single_model_mae[2][0], cluster)], globals()["{}_test_{}".format(single_model_mae[2][0], cluster)] = get_stacking_base_datasets(getattr(mod, "{}_model_tuned_{}".format(single_model_mae[2][0], cluster)), x_train_n, y_train_n, x_test_n, 5) 
                                                                                                            

  stack_final_x_train = np.concatenate((getattr(mod, "{}_train_{}".format(single_model_mae[0][0], cluster)), getattr(mod, "{}_train_{}".format(single_model_mae[2][0], cluster))), axis=1)
  stack_final_x_test = np.concatenate((getattr(mod, "{}_test_{}".format(single_model_mae[0][0], cluster)), getattr(mod, "{}_test_{}".format(single_model_mae[2][0], cluster))), axis=1)

  globals()["meta_model_{}_{}".format(single_model_mae[1][0], cluster)] = getattr(mod, "{}_model_tuned_{}".format(single_model_mae[1][0], cluster))

  getattr(mod, "meta_model_{}_{}".format(single_model_mae[1][0], cluster)).fit(stack_final_x_train, y_train)
  globals()["stack_pred_{}_{}".format(single_model_mae[1][0], cluster)] = getattr(mod, "meta_model_{}_{}".format(single_model_mae[1][0], cluster)).predict(stack_final_x_test)
  stacking_list["{}".format(single_model_mae[1][0])] =  mean_absolute_error(y_test, getattr(mod, "stack_pred_{}_{}".format(single_model_mae[1][0], cluster)))


  # 세번째 경우
  globals()["{}_train_{}".format(single_model_mae[1][0], cluster)], globals()["{}_test_{}".format(single_model_mae[1][0], cluster)] = get_stacking_base_datasets(getattr(mod, "{}_model_tuned_{}".format(single_model_mae[1][0], cluster)), x_train_n, y_train_n, x_test_n, 5)
                                                                                                            
  globals()["{}_train_{}".format(single_model_mae[2][0], cluster)], globals()["{}_test_{}".format(single_model_mae[2][0], cluster)] = get_stacking_base_datasets(getattr(mod, "{}_model_tuned_{}".format(single_model_mae[2][0], cluster)), x_train_n, y_train_n, x_test_n, 5) 
                                                                                                            

  stack_final_x_train = np.concatenate((getattr(mod, "{}_train_{}".format(single_model_mae[1][0], cluster)), getattr(mod, "{}_train_{}".format(single_model_mae[2][0], cluster))), axis=1)
  stack_final_x_test = np.concatenate((getattr(mod, "{}_test_{}".format(single_model_mae[1][0], cluster)), getattr(mod, "{}_test_{}".format(single_model_mae[2][0], cluster))), axis=1)

  globals()["meta_model_{}_{}".format(single_model_mae[0][0], cluster)] = getattr(mod, "{}_model_tuned_{}".format(single_model_mae[0][0], cluster))

  getattr(mod, "meta_model_{}_{}".format(single_model_mae[0][0], cluster)).fit(stack_final_x_train, y_train)
  globals()["stack_pred_{}_{}".format(single_model_mae[0][0], cluster)] = getattr(mod, "meta_model_{}_{}".format(single_model_mae[0][0], cluster)).predict(stack_final_x_test)
  stacking_list["{}".format(single_model_mae[0][0])] =  mean_absolute_error(y_test, getattr(mod, "stack_pred_{}_{}".format(single_model_mae[0][0], cluster)))

  stacking_model_mae = sorted(stacking_list.items(), key = lambda item: item[1])

# 단일 모델과 stacking 모델의 MAE 값을 비교하여 작은 값으로 모델링 결과 반환
  a = single_model_mae[0][1]
  b = stacking_model_mae[0][1]

  if a < b:
    print("클러스터 {}의 최적 모델은 단일 모델 {} : (test MAE값) {}".format(cluster, single_model_mae[0][0], single_model_mae[0][1]))
    globals()["best_model_{}".format(cluster)]  = single_model_mae[0][0]
    globals()["test_mae_{}".format(cluster)] = single_model_mae[0][1]
    globals()["best_pred_{}".format(cluster)] = getattr(mod, "single_pred_{}_{}".format(single_model_mae[0][0], cluster))
  else:
    print("클러스터 {}의 최적 모델은 stacking meta 모델 {} : (test MAE값) {}".format(cluster, stacking_model_mae[0][0], stacking_model_mae[0][1]))
    globals()["best_model_{}".format(cluster)] = stacking_model_mae[0][0]
    globals()["test_mae_{}".format(cluster)] = stacking_model_mae[0][1]
    globals()["best_pred_{}".format(cluster)] = getattr(mod, "stack_pred_{}_{}".format(stacking_model_mae[0][0], cluster))


In [26]:
# 단일 모델과 stacking 모델의 MAE 값을 비교하여 작은 값으로 모델링 결과 반환
for cluster in cluster_df:
  get_optimal_model(cluster)

클러스터 0.0의 최적 모델은 단일 모델 SVR : (test MAE값) 0.0988
클러스터 1.0의 최적 모델은 단일 모델 SVR : (test MAE값) 0.0131
클러스터 2.0의 최적 모델은 단일 모델 SVR : (test MAE값) 0.0674
클러스터 3.0의 최적 모델은 단일 모델 SVR : (test MAE값) 79.5137
클러스터 4.0의 최적 모델은 단일 모델 SVR : (test MAE값) 0.0994
클러스터 5.0의 최적 모델은 단일 모델 SVR : (test MAE값) 0.073
클러스터 6.0의 최적 모델은 단일 모델 SVR : (test MAE값) 0.0495
클러스터 7.0의 최적 모델은 단일 모델 SVR : (test MAE값) 2.105
클러스터 8.0의 최적 모델은 단일 모델 SVR : (test MAE값) 0.075
클러스터 9.0의 최적 모델은 단일 모델 SVR : (test MAE값) 0.0894
클러스터 10의 최적 모델은 단일 모델 SVR : (test MAE값) 1.0727


### 모델 학습 결과에 대한 예측

In [27]:
for cluster in cluster_df:
  x_train, y_train, x_test, y_test = get_train_test_set(cluster)
  globals()['pred_{}'.format(cluster)]= getattr(mod, "stack_pred_{}_{}".format(getattr(mod, "best_model_{}".format(cluster)), cluster))

In [28]:
# test dataset에 대한 예측값 
cluster_pred_df = pd.DataFrame()
for cluster in cluster_df:
  cluster_pred_df[str(cluster)] = getattr(mod, 'best_pred_{}'.format(cluster))

In [29]:
cluster_pred_df

,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10
0,166.10037,7594.97361,1398.07139,81.73906,39.09945,1111.06868,5766.04072,1810.56301,9820.06514,998.07880,150.07448
1,169.09884,6602.99179,1055.07792,100.49231,48.09938,990.07277,5343.04579,1922.28884,9057.07069,479.09027,137.57177
2,182.09882,6573.99264,1007.08060,86.76966,50.09938,974.07313,4754.05148,1299.35486,8466.07533,426.09133,118.54490
3,135.09947,6091.00082,1051.07609,91.28171,38.09967,992.07243,4675.05255,1642.15016,7725.08070,402.09101,129.39766
4,115.09962,6502.00657,1089.07487,109.77862,109.09892,622.08434,5433.04435,2096.67929,6686.08861,432.09104,100.55541
5,106.09922,4373.03115,801.08279,76.35226,77.09923,549.08619,4139.05763,2967.92142,6725.08780,256.09482,112.60687
6,160.09669,6513.99801,1295.06579,390.78756,65.09967,914.07348,4664.05183,2563.56521,8532.07400,491.08842,170.05508
7,225.09772,7195.98537,4515.97876,2621.65727,51.09927,1650.05270,5164.04719,2663.51074,9958.06363,498.08896,156.88006
8,204.09837,7654.98059,1952.05728,1794.95471,50.09943,1041.07240,4896.05034,1421.16306,9073.07050,525.08846,206.01491
9,172.09910,7384.98590,1687.06589,31959.89892,42.09950,979.07418,4594.05268,1373.43767,8598.07386,461.09059,281.57189


In [30]:
cluster_pred_df.to_csv('./모델링/예측값/cluster_예측값.csv', encoding = 'utf-8', index = False)